In [1]:
import torch
import torch.nn as nn
import math
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### 加载训练好的本地模型

In [3]:
class TransformerClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, num_encoder_layers, ff_dim, num_classes,
                     max_len=512, dropout_rate=0.1):
        """
        初始化 Transformer 分类器
        
        Args:
            vocab_size (int): 词汇表大小（tokenizer.vocab_size）。
            embed_dim (int): 词嵌入和 Transformer 的维度（d_model）。
            num_heads (int): 多头注意力机制的头数，必须能整除 embed_dim。
            num_encoder_layers (int): Transformer Encoder 的层数。
            ff_dim (int): 前馈网络中间层的维度（通常为 embed_dim 的 2-4 倍）。
            num_classes (int): 分类任务的类别数（2 表示正/负）。
            max_len (int): 最大序列长度，用于位置嵌入。
            dropout_rate (float): Dropout 比率，用于正则化。
        """
        super().__init__()
        self.embed_dim = embed_dim
        # 词嵌入层，将 token ID 映射到 embed_dim 维向量
        self.token_embedding = nn.Embedding(vocab_size, embed_dim)
        # 可学习的位置嵌入，为每个位置生成 embed_dim 维向量
        self.positional_embedding = nn.Embedding(max_len, embed_dim)

        # 定义单个 Transformer Encoder 层
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,              # 模型维度
            nhead=num_heads,                # 注意力头数
            dim_feedforward=ff_dim,         # 前馈网络中间层维度
            dropout=dropout_rate,            # Dropout 比率
            batch_first=True,               # 输入/输出形状为 (batch, seq, feature)，适配常见数据格式
            activation='gelu'               # 使用 GELU 激活函数，相比 ReLU 更平滑，有助于梯度流动
        )

        # 堆叠多个 Transformer Encoder 层
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_encoder_layers,
            norm=nn.LayerNorm(embed_dim)    # 显式添加 LayerNorm，规范化输出
        )

        self.dropout = nn.Dropout(dropout_rate)
        
        # 分类头：将 [CLS] token 的输出（embed_dim 维）映射到 num_classes 维
        self.fc_out = nn.Linear(embed_dim, num_classes)

        self.max_len = max_len # 存储最大序列长度，供位置编码使用

        self._init_weights()

    def _init_weights(self):
        """
        初始化模型权重，使用 Xavier Uniform 初始化，适合 Transformer 模型。
        避免初始权重过大或过小，加速收敛。
        """
        for p in self.parameters():
            if p.dim() > 1:  # 仅对二维以上参数（如线性层、嵌入层）应用
                nn.init.xavier_uniform_(p)
            # 对嵌入层可额外应用正态初始化
            elif p.dim() == 2 and 'embedding' in p.name:
                nn.init.normal_(p, mean=0.0, std=0.02)

    def forward(self, input_ids, attention_mask):
        """
        前向传播，处理输入序列并输出分类 logits。

        Args:
            input_ids (torch.Tensor): 形状 (batch_size, seq_len)，词的 ID。
            attention_mask (torch.Tensor): 形状 (batch_size, seq_len)，1 表示有效 token，0 表示 padding。

        Returns:
            torch.Tensor: 形状 (batch_size, num_classes)，分类 logits。
        """
        seq_len = input_ids.size(1)  # 获取序列长度

        # 1. 词嵌入
        token_embeds = self.token_embedding(input_ids)  # (batch_size, seq_len, embed_dim)
        token_embeds = token_embeds * math.sqrt(self.embed_dim) # 缩放嵌入，稳定训练

        # 2. 位置编码
        # 生成位置索引：(batch_size, seq_len)，每个样本重复 0 到 seq_len-1
        positions = torch.arange(0, seq_len, device=input_ids.device).unsqueeze(0).repeat(input_ids.size(0), 1)
        position_embeds = self.positional_embedding(positions)  # (batch_size, seq_len, embed_dim)
        
        # 词嵌入与位置嵌入相加
        x = token_embeds + position_embeds
        x = self.dropout(x)  # 在嵌入后应用 Dropout，增强鲁棒性

        # Transformer Encoder需要 src_key_padding_mask
        # attention_mask: 1是token, 0是padding.
        # src_key_padding_mask: True表示该位置是padding, 需要被mask掉.
        src_key_padding_mask = (attention_mask == 0)  # (batch_size, seq_len)

        # 3. Transformer Encoder
        # 输入形状: (batch_size, seq_len, embed_dim)
        encoder_output = self.transformer_encoder(x, src_key_padding_mask=src_key_padding_mask)
        # encoder_output shape: (batch_size, seq_len, embed_dim)

        # 4. 分类
        # 通常使用第一个token ([CLS] token)的输出来进行分类
        cls_output = encoder_output[:, 0, :]  # (batch_size, embed_dim)
        # 或者，可以对所有token的输出进行平均池化或最大池化
        # cls_output = encoder_output.mean(dim=1) # 平均池化

        cls_output = self.dropout(cls_output)
        logits = self.fc_out(cls_output)  # (batch_size, num_classes)

        return logits


# 加载Bert 的分词器
tokenizer_path = '../models/3_Transformer_Sentiment_Classification/bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
    
# 定义模型超参数
VOCAB_SIZE = tokenizer.vocab_size  # 从之前加载的 BERT 分词器获取
EMBED_DIM = 256                   # 嵌入维度，较小以减少计算量（BERT 常用 768）
NUM_HEADS = 8                     # 多头注意力头数，需满足 embed_dim % num_heads == 0
NUM_ENCODER_LAYERS = 4            # Encoder 层数，平衡性能与计算成本
FF_DIM = 512                      # 前馈网络中间层维度，通常为 embed_dim 的 2-4 倍
NUM_CLASSES = 2                   # 分类任务的类别数（正/负情感）
DROPOUT_RATE = 0.1                # Dropout 比率，防止过拟合
MAX_LEN = 128


model = TransformerClassifier(
    vocab_size=VOCAB_SIZE,
    embed_dim=EMBED_DIM,
    num_heads=NUM_HEADS,
    num_encoder_layers=NUM_ENCODER_LAYERS,
    ff_dim=FF_DIM,
    num_classes=NUM_CLASSES,
    max_len=MAX_LEN, # 从之前的配置中获取
    dropout_rate=DROPOUT_RATE
)
model = model.to(device)

model_path = '../models/3_Transformer_Sentiment_Classification/model_weights.pth'

# 加载模型参数
model.load_state_dict(torch.load(model_path))

# 将模型设置为评估模式
model.eval()

C:\Users\PC\AppData\Local\Temp\ipykernel_27116\44386423.py:140: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


TransformerClassifier(
  (token_embedding): Embedding(21128, 256)
  (positional_embedding): Embedding(128, 256)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (linear1): Linear(in_features=256, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=512, out_features=256, bias=True)
        (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
    (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (fc_out): Linear(in_features=256, out_features=2, bias=True)

## 模型推理

In [4]:
def predict_sentiment(text, max_len=MAX_LEN):
    #  文本预处理
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,  # 添加 [CLS] 和 [SEP]
        max_length=max_len,
        padding='max_length',     # 填充到 max_len
        truncation=True,          # 截断超长文本
        return_tensors='pt',      # 返回 PyTorch 张量
        return_attention_mask=True
    )
    input_ids = encoding['input_ids'].to(device)          # (1, max_len)
    attention_mask = encoding['attention_mask'].to(device)  # (1, max_len)
    # 模型推理
    with torch.no_grad():  # 禁用梯度计算，节省内存
        logits = model(input_ids, attention_mask)  # (1, num_classes)
        pred = torch.argmax(logits, dim=-1).item()  # 预测类别 (0 或 1)

    # 转换标签
    return '正面' if pred == 1 else '负面'

In [5]:
pos_review = "虽没有第一部“我命由我不由天”的惊艳金句，但更多了些“怎能不知道这世间的规则，由谁所定？”的结构性思考，无量仙翁的“个体失范代替制度失范”真是最佳切口。狠狠期待第三部！"
neg_review = "我觉得中国这些人拍点电影，啥时候变成这种短视频短剧形式的切片合集了？一点点深度也没有了？太快餐了"
print(f'Review : {pos_review}\nsentiment : {predict_sentiment(pos_review)}')
print(f'Review : {neg_review}\nsentiment : {predict_sentiment(neg_review)}')

Review : 虽没有第一部“我命由我不由天”的惊艳金句，但更多了些“怎能不知道这世间的规则，由谁所定？”的结构性思考，无量仙翁的“个体失范代替制度失范”真是最佳切口。狠狠期待第三部！
sentiment : 正面
Review : 我觉得中国这些人拍点电影，啥时候变成这种短视频短剧形式的切片合集了？一点点深度也没有了？太快餐了
sentiment : 负面


D:\develop\anaconda\envs\pytorch\lib\site-packages\torch\nn\modules\transformer.py:409: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\NestedTensorImpl.cpp:180.)
  output = torch._nested_tensor_from_mask(output, src_key_padding_mask.logical_not(), mask_check=False)
D:\develop\anaconda\envs\pytorch\lib\site-packages\torch\nn\modules\transformer.py:720: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  return torch._transformer_encoder_layer_fwd(
